Importing The Twitter dataset

In [ ]:
#importing the dataset
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#Extracting the compressed dataset
from zipfile import ZipFile
dataset = 'sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Done')

Done


Importing the Libraries


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Load the dataset


In [ ]:
data = pd.read_csv(r'/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
# Basic data checking
data.shape

(1599999, 6)

In [ ]:
# Printing the first 5 rows of the dataset
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# as we can see padas not reading the column name properly so renaming the columns and reading the data again according to the dataset
column_name = ['target','ids','date','flag','user','text']
data = pd.read_csv(r'/content/training.1600000.processed.noemoticon.csv',names = column_name, encoding = 'ISO-8859-1')
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# checking for missing values
data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
data['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
# changing the labels 4 to 1
data.replace({'target':{4:1}},inplace = True)

In [ ]:
data['target'].value_counts()

,count
target,
0,800000
1,800000


0 ----> Negative Tweet

1 ----> Positive Tweet

**Stemming**

Stemming is the process of reducing the word to its root word

Example = actor,actress,acting = act

In [ ]:
port_stem = PorterStemmer()

def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
data['stemmed_content'] = data['text'].apply(stemming)

In [ ]:
data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [32]:
print(data['text'][0])

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D


Seperating the data for training


In [ ]:
x = data['stemmed_content'].values
y = data['target'].values

In [ ]:
X_train , X_test , Y_train, Y_test = train_test_split(x,y , test_size=0.2,stratify = y, random_state=4)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


Training the Ml model (Logiastic Regression)

In [ ]:
model = LogisticRegression(max_iter = 1000)

In [23]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [24]:
# Accuracy on training data
x_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction,Y_train)

In [25]:
print("Accuracy: ",training_data_accuracy)

Accuracy:  0.8100171875


In [26]:
# Accuracy on testing data
x_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(x_test_prediction,Y_test)
print("Accuracy on test data: ",testing_data_accuracy)

Accuracy on test data:  0.779396875


Saving the Trained Model

In [27]:
import pickle


In [28]:
filename = 'twitter_sentiment.pkl'
pickle.dump(model,open(filename,'wb'))

with open('twitter_sentiment.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [29]:
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

**Loading the model


In [35]:
#LOading the model


with open('twitter_sentiment.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)




In [36]:
def preprocess_text(text):
    # Remove unwanted characters and lowercase the text
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()

    # Stemming and removing stopwords
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    return ' '.join(text)

tweet = input("Enter a tweet: ")
if tweet:
  preprocessed_tweet = preprocess_text(tweet)
  # Transform the tweet using the vectorizer
  tweet_vectorized = vectorizer.transform([preprocessed_tweet])
  # Predict the sentiment
  prediction = model.predict(tweet_vectorized)
  sentiment = "Positive" if prediction == 1 else "Negative"
  # Display the prediction
  print(f"The sentiment of the tweet is **{sentiment}**.")
else:
  print("Please enter a tweet to analyze.")




Enter a tweet: Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D.
The sentiment of the tweet is **Negative**.
